## BUSINESS UNDERSTANDING

# Overview

Road accidents are a common occurrence in our daily lives, and it is likely that you or someone you know has been involved in one, hopefully without severe consequences. The ultimate goal is to reduce the number of road accidents. We will work with the Chicago Car Crash dataset to identify trends and predict the major causes of road accidents. By analyzing this data, we aim to develop strategies to reduce the frequency and severity of road accidents.

# Business Problem 

For this project our stakeholders are NHTSA (National Highway Traffic Safety Administration) which is the governing body in the United States of America that deals with road safety. Our objectives for our stakeholders is to explain to them:
    1. Predict the major cause of road accidents in Chicago
    2. Identify the common trends of road accidents
    3. Analyze the Temporal Distribution of Accidents, identify peak times and days of the week when accidents are most likely to occur.



## DATA UNDERSTANDING

For this project the dataset we will be using is the Chicago Car Crashes Dataset. This data set includes two csv files, Traffic Crashes - People and Traffic Crashes - Vehicles